## IMF WEO

version 2.1 - 18 mai 2017

### What it does
This script transfrom IMF WEO dataset and create json files. In order to proceed, we will follow these steps:
+ **Process** WEO dataset (transpose)
+ **Compute world, regional and income group averages**, this way:
    + merge all CSVs into a unique dataset
    + compute group averages for each years
+ **Write json files which contain the data and meta we want** (begining and end of the serie for each indicator/ countries) and other meta information such as: indicator label, source, download date, Country meta, etc.


## 1. Process WEO dataset

In [3]:
import pandas as pd
from pandas_datareader import data, wb
from pandas import Series, DataFrame, concat
import numpy as np
from datetime import datetime
import sys,os,os.path
import winsound

#This dictionnary associate a country name to an iso3 code 
country_dict = {"Yemen":"YEM","Arab World":"ARB", "Central Europe and the Baltics":"CEB", "Caribbean small states":"CSS", "East Asia & Pacific":"EAP", "East Asia & Pacific (all income levels)":"EAS", "Europe & Central Asia":"ECA", "Europe & Central Asia (all income levels)":"ECS", "Euro area":"EMU", "European Union":"EUU", "Fragile and conflict affected situations":"FCS", "High income":"HIC", "Heavily indebted poor countries (HIPC)":"HPC", "Latin America & Caribbean":"LAC", "Latin America & Caribbean (all income levels)":"LCN", "Least developed countries: UN classification":"LDC", "Low income":"LIC", "Lower middle income":"LMC", "Low & middle income":"LMY", "Middle East & North Africa (all income levels)":"MEA", "Middle income":"MIC", "Middle East & North Africa":"MNA", "North America":"NAC", "High income: nonOECD":"NOC", "High income: OECD":"OEC", "OECD members":"OED", "Other small states":"OSS", "Pacific island small states":"PSS", "South Asia":"SAS", "Sub-Saharan Africa":"SSA", "Sub-Saharan Africa (all income levels)":"SSF", "Small states":"SST", "Upper middle income":"UMC", "World":"WLD","Cote d'Ivoire":"CIV","Curacao":"CUW","Sao Tome and Principe":"STP","American Samoa":"ASM","Australia":"AUS","Brunei Darussalam":"BRN","China":"CHN","Fiji":"FJI","Micronesia, Fed. Sts.":"FSM","Guam":"GUM","Hong Kong SAR, China":"HKG","Indonesia":"IDN","Japan":"JPN","Cambodia":"KHM","Kiribati":"KIR","Korea, Rep.":"KOR","Lao PDR":"LAO","Macao SAR, China":"MAC","Marshall Islands":"MHL","Myanmar":"MMR","Mongolia":"MNG","Northern Mariana Islands":"MNP","Malaysia":"MYS","New Caledonia":"NCL","New Zealand":"NZL","Philippines":"PHL","Palau":"PLW","Papua New Guinea":"PNG","Korea, Dem. Rep.":"PRK","French Polynesia":"PYF","Singapore":"SGP","Solomon Islands":"SLB","Thailand":"THA","Timor-Leste":"TLS","Tonga":"TON","Tuvalu":"TUV","Taiwan, China":"TWN","Vietnam":"VNM","Vanuatu":"VUT","Samoa":"WSM","Albania":"ALB","Andorra":"AND","Armenia":"ARM","Austria":"AUT","Azerbaijan":"AZE","Belgium":"BEL","Bulgaria":"BGR","Bosnia and Herzegovina":"BIH","Belarus":"BLR","Switzerland":"CHE","Channel Islands":"CHI","Cyprus":"CYP","Czech Republic":"CZE","Germany":"DEU","Denmark":"DNK","Spain":"ESP","Estonia":"EST","Finland":"FIN","France":"FRA","Faeroe Islands":"FRO","United Kingdom":"GBR","Georgia":"GEO","Greece":"GRC","Greenland":"GRL","Croatia":"HRV","Hungary":"HUN","Isle of Man":"IMN","Ireland":"IRL","Iceland":"ISL","Italy":"ITA","Kazakhstan":"KAZ","Kyrgyz Republic":"KGZ","Liechtenstein":"LIE","Lithuania":"LTU","Luxembourg":"LUX","Latvia":"LVA","Monaco":"MCO","Moldova":"MDA","Macedonia, FYR":"MKD","Montenegro":"MNE","Netherlands":"NLD","Norway":"NOR","Poland":"POL","Portugal":"PRT","Romania":"ROU","Russian Federation":"RUS","San Marino":"SMR","Serbia":"SRB","Slovak Republic":"SVK","Slovenia":"SVN","Sweden":"SWE","Tajikistan":"TJK","Turkmenistan":"TKM","Turkey":"TUR","Ukraine":"UKR","Uzbekistan":"UZB","Aruba":"ABW","Argentina":"ARG","Antigua and Barbuda":"ATG","Bahamas, The":"BHS","Belize":"BLZ","Bolivia":"BOL","Brazil":"BRA","Barbados":"BRB","Chile":"CHL","Colombia":"COL","Costa Rica":"CRI","Cuba":"CUB","Curaçao":"CUW","Cayman Islands":"CYM","Dominica":"DMA","Dominican Republic":"DOM","Ecuador":"ECU","Grenada":"GRD","Guatemala":"GTM","Guyana":"GUY","Honduras":"HND","Haiti":"HTI","Jamaica":"JAM","St. Kitts and Nevis":"KNA","St. Lucia":"LCA","St. Martin (French part)":"MAF","Mexico":"MEX","Nicaragua":"NIC","Panama":"PAN","Peru":"PER","Puerto Rico":"PRI","Paraguay":"PRY","El Salvador":"SLV","Suriname":"SUR","Sint Maarten (Dutch part)":"SXM","Turks and Caicos Islands":"TCA","Trinidad and Tobago":"TTO","Uruguay":"URY","St. Vincent and the Grenadines":"VCT","Venezuela, RB":"VEN","Virgin Islands (U.S.)":"VIR","United Arab Emirates":"ARE","Bahrain":"BHR","Djibouti":"DJI","Algeria":"DZA","Egypt, Arab Rep.":"EGY","Iran, Islamic Rep.":"IRN","Iraq":"IRQ","Israel":"ISR","Jordan":"JOR","Kuwait":"KWT","Lebanon":"LBN","Libya":"LBY","Morocco":"MAR","Malta":"MLT","Oman":"OMN","West Bank and Gaza":"PSE","Qatar":"QAT","Saudi Arabia":"SAU","Syrian Arab Republic":"SYR","Tunisia":"TUN","Bermuda":"BMU","Canada":"CAN","United States":"USA","Afghanistan":"AFG","Bangladesh":"BGD","Bhutan":"BTN","India":"IND","Sri Lanka":"LKA","Maldives":"MDV","Nepal":"NPL","Pakistan":"PAK","Angola":"AGO","Burundi":"BDI","Benin":"BEN","Burkina Faso":"BFA","Botswana":"BWA","Central African Republic":"CAF","Côte d'Ivoire":"CIV","Cameroon":"CMR","Congo, Dem. Rep.":"COD","Congo, Rep.":"COG","Comoros":"COM","Cabo Verde":"CPV","Eritrea":"ERI","Ethiopia":"ETH","Gabon":"GAB","Ghana":"GHA","Guinea":"GIN","Gambia, The":"GMB","Guinea-Bissau":"GNB","Equatorial Guinea":"GNQ","Kenya":"KEN","Liberia":"LBR","Lesotho":"LSO","Madagascar":"MDG","Mali":"MLI","Mozambique":"MOZ","Mauritania":"MRT","Mauritius":"MUS","Malawi":"MWI","Namibia":"NAM","Niger":"NER","Nigeria":"NGA","Rwanda":"RWA","Sudan":"SDN","Senegal":"SEN","Sierra Leone":"SLE","Somalia":"SOM","South Sudan":"SSD","São Tomé and Principe":"STP","Swaziland":"SWZ","Seychelles":"SYC","Chad":"TCD","Togo":"TGO","Tanzania":"TZA","Uganda":"UGA","South Africa":"ZAF","Zambia":"ZMB","Zimbabwe":"ZWE"}

#Region dictionnary that affect to a country iso3 its income group, region group, and worl:
region_dict= {"ASM":"UMC,EAS,WLD","AUS":"OEC,EAS,WLD","BRN":"NOC,EAS,WLD","CHN":"UMC,EAS,WLD","FJI":"UMC,EAS,WLD","FSM":"LMC,EAS,WLD","GUM":"NOC,EAS,WLD","HKG":"NOC,EAS,WLD","IDN":"LMC,EAS,WLD","JPN":"OEC,EAS,WLD","KHM":"LIC,EAS,WLD","KIR":"LMC,EAS,WLD","KOR":"OEC,EAS,WLD","LAO":"LMC,EAS,WLD","MAC":"NOC,EAS,WLD","MHL":"UMC,EAS,WLD","MMR":"LMC,EAS,WLD","MNG":"UMC,EAS,WLD","MNP":"NOC,EAS,WLD","MYS":"UMC,EAS,WLD","NCL":"NOC,EAS,WLD","NZL":"OEC,EAS,WLD","PHL":"LMC,EAS,WLD","PLW":"UMC,EAS,WLD","PNG":"LMC,EAS,WLD","PRK":"LIC,EAS,WLD","PYF":"NOC,EAS,WLD","SGP":"NOC,EAS,WLD","SLB":"LMC,EAS,WLD","THA":"UMC,EAS,WLD","TLS":"LMC,EAS,WLD","TON":"UMC,EAS,WLD","TUV":"UMC,EAS,WLD","TWN":"NOC,EAS,WLD","VNM":"LMC,EAS,WLD","VUT":"LMC,EAS,WLD","WSM":"LMC,EAS,WLD","":"LMC,ECS,WLD","ALB":"UMC,ECS,WLD","AND":"NOC,ECS,WLD","ARM":"LMC,ECS,WLD","AUT":"OEC,ECS,WLD","AZE":"UMC,ECS,WLD","BEL":"OEC,ECS,WLD","BGR":"UMC,ECS,WLD","BIH":"UMC,ECS,WLD","BLR":"UMC,ECS,WLD","CHE":"OEC,ECS,WLD","CHI":"NOC,ECS,WLD","CYP":"NOC,ECS,WLD","CZE":"OEC,ECS,WLD","DEU":"OEC,ECS,WLD","DNK":"OEC,ECS,WLD","ESP":"OEC,ECS,WLD","EST":"OEC,ECS,WLD","FIN":"OEC,ECS,WLD","FRA":"OEC,ECS,WLD","FRO":"NOC,ECS,WLD","GBR":"OEC,ECS,WLD","GEO":"LMC,ECS,WLD","GRC":"OEC,ECS,WLD","GRL":"NOC,ECS,WLD","HRV":"NOC,ECS,WLD","HUN":"OEC,ECS,WLD","IMN":"NOC,ECS,WLD","IRL":"OEC,ECS,WLD","ISL":"OEC,ECS,WLD","ITA":"OEC,ECS,WLD","KAZ":"UMC,ECS,WLD","KGZ":"LMC,ECS,WLD","LIE":"NOC,ECS,WLD","LTU":"NOC,ECS,WLD","LUX":"OEC,ECS,WLD","LVA":"NOC,ECS,WLD","MCO":"NOC,ECS,WLD","MDA":"LMC,ECS,WLD","MKD":"UMC,ECS,WLD","MNE":"UMC,ECS,WLD","NLD":"OEC,ECS,WLD","NOR":"OEC,ECS,WLD","POL":"OEC,ECS,WLD","PRT":"OEC,ECS,WLD","ROU":"UMC,ECS,WLD","RUS":"NOC,ECS,WLD","SMR":"NOC,ECS,WLD","SRB":"UMC,ECS,WLD","SVK":"OEC,ECS,WLD","SVN":"OEC,ECS,WLD","SWE":"OEC,ECS,WLD","TJK":"LMC,ECS,WLD","TKM":"UMC,ECS,WLD","TUR":"UMC,ECS,WLD","UKR":"LMC,ECS,WLD","UZB":"LMC,ECS,WLD","ABW":"NOC,LCN,WLD","ARG":"NOC,LCN,WLD","ATG":"NOC,LCN,WLD","BHS":"NOC,LCN,WLD","BLZ":"UMC,LCN,WLD","BOL":"LMC,LCN,WLD","BRA":"UMC,LCN,WLD","BRB":"NOC,LCN,WLD","CHL":"OEC,LCN,WLD","COL":"UMC,LCN,WLD","CRI":"UMC,LCN,WLD","CUB":"UMC,LCN,WLD","CUW":"NOC,LCN,WLD","CYM":"NOC,LCN,WLD","DMA":"UMC,LCN,WLD","DOM":"UMC,LCN,WLD","ECU":"UMC,LCN,WLD","GRD":"UMC,LCN,WLD","GTM":"LMC,LCN,WLD","GUY":"LMC,LCN,WLD","HND":"LMC,LCN,WLD","HTI":"LIC,LCN,WLD","JAM":"UMC,LCN,WLD","KNA":"NOC,LCN,WLD","LCA":"UMC,LCN,WLD","MAF":"NOC,LCN,WLD","MEX":"UMC,LCN,WLD","NIC":"LMC,LCN,WLD","PAN":"UMC,LCN,WLD","PER":"UMC,LCN,WLD","PRI":"NOC,LCN,WLD","PRY":"UMC,LCN,WLD","SLV":"LMC,LCN,WLD","SUR":"UMC,LCN,WLD","SXM":"NOC,LCN,WLD","TCA":"NOC,LCN,WLD","TTO":"NOC,LCN,WLD","URY":"NOC,LCN,WLD","VCT":"UMC,LCN,WLD","VEN":"NOC,LCN,WLD","VIR":"NOC,LCN,WLD","ARE":"NOC,MEA,WLD","BHR":"NOC,MEA,WLD","DJI":"LMC,MEA,WLD","DZA":"UMC,MEA,WLD","EGY":"LMC,MEA,WLD","IRN":"UMC,MEA,WLD","IRQ":"UMC,MEA,WLD","ISR":"OEC,MEA,WLD","JOR":"UMC,MEA,WLD","KWT":"NOC,MEA,WLD","LBN":"UMC,MEA,WLD","LBY":"UMC,MEA,WLD","MAR":"LMC,MEA,WLD","MLT":"NOC,MEA,WLD","OMN":"NOC,MEA,WLD","PSE":"LMC,MEA,WLD","QAT":"NOC,MEA,WLD","SAU":"NOC,MEA,WLD","SYR":"LMC,MEA,WLD","TUN":"UMC,MEA,WLD","YEM":"LMC,MEA,WLD","BMU":"NOC,NAC,WLD","CAN":"OEC,NAC,WLD","USA":"OEC,NAC,WLD","AFG":"LIC,SAS,WLD","BGD":"LMC,SAS,WLD","BTN":"LMC,SAS,WLD","IND":"LMC,SAS,WLD","LKA":"LMC,SAS,WLD","MDV":"UMC,SAS,WLD","NPL":"LIC,SAS,WLD","PAK":"LMC,SAS,WLD","AGO":"UMC,SSF,WLD","BDI":"LIC,SSF,WLD","BEN":"LIC,SSF,WLD","BFA":"LIC,SSF,WLD","BWA":"UMC,SSF,WLD","CAF":"LIC,SSF,WLD","CIV":"LMC,SSF,WLD","CMR":"LMC,SSF,WLD","COD":"LIC,SSF,WLD","COG":"LMC,SSF,WLD","COM":"LIC,SSF,WLD","CPV":"LMC,SSF,WLD","ERI":"LIC,SSF,WLD","ETH":"LIC,SSF,WLD","GAB":"UMC,SSF,WLD","GHA":"LMC,SSF,WLD","GIN":"LIC,SSF,WLD","GMB":"LIC,SSF,WLD","GNB":"LIC,SSF,WLD","GNQ":"NOC,SSF,WLD","KEN":"LMC,SSF,WLD","LBR":"LIC,SSF,WLD","LSO":"LMC,SSF,WLD","MDG":"LIC,SSF,WLD","MLI":"LIC,SSF,WLD","MOZ":"LIC,SSF,WLD","MRT":"LMC,SSF,WLD","MUS":"UMC,SSF,WLD","MWI":"LIC,SSF,WLD","NAM":"UMC,SSF,WLD","NER":"LIC,SSF,WLD","NGA":"LMC,SSF,WLD","RWA":"LIC,SSF,WLD","SDN":"LMC,SSF,WLD","SEN":"LMC,SSF,WLD","SLE":"LIC,SSF,WLD","SOM":"LIC,SSF,WLD","SSD":"LIC,SSF,WLD","STP":"LMC,SSF,WLD","SWZ":"LMC,SSF,WLD","SYC":"NOC,SSF,WLD","TCD":"LIC,SSF,WLD","TGO":"LIC,SSF,WLD","TZA":"LIC,SSF,WLD","UGA":"LIC,SSF,WLD","ZAF":"UMC,SSF,WLD","ZMB":"LMC,SSF,WLD","ZWE":"LIC,SSF,WLD"}

iso3list_weo=["AFG","ALB","DZA","AGO","ATG","ARG","ARM","AUS","AUT","AZE","BHR","BGD","BRB","BLR","BEL","BLZ","BEN","BTN","BOL","BIH","BWA","BRA","BRN","BGR","BFA","BDI","CPV","KHM","CMR","CAN","CAF","TCD","CHL","CHN","COL","COM","CRI","HRV","CYP","CZE","DNK","DJI","DMA","DOM","ECU","SLV","GNQ","ERI","EST","ETH","FJI","FIN","FRA","GAB","GEO","DEU","GHA","GRC","GRD","GTM","GIN","GNB","GUY","HTI","HND","HUN","ISL","IND","IDN","IRQ","IRL","ISR","ITA","JAM","JPN","JOR","KAZ","KEN","KIR","KWT","KGZ","LVA","LBN","LSO","LBR","LBY","LTU","LUX","MDG","MWI","MYS","MDV","MLI","MLT","MHL","MRT","MUS","MEX","MDA","MNG","MNE","MAR","MOZ","MMR","NAM","NPL","NLD","NZL","NIC","NER","NGA","NOR","OMN","PAK","PLW","PAN","PNG","PRY","PER","PHL","POL","PRT","PRI","QAT","ROU","RWA","WSM","SMR","SAU","SEN","SRB","SYC","SLE","SGP","SVK","SVN","SLB","ZAF","SSD","ESP","LKA","KNA","LCA","VCT","SDN","SUR","SWZ","SWE","CHE","TJK","TZA","THA","TLS","TGO","TON","TTO","TUN","TUR","TKM","TUV","UGA","UKR","ARE","GBR","USA","URY","UZB","VUT","YEM","VNM","ZMB","ZWE"]

indicator_list=['NID_NGDP','PPPGDP','BCA_NGDPD','BCA','LE','TX_RPCH','TXG_RPCH','NGDP_FY','NGDP_D','NGDP_RPCH',\
                'NGDP_R','NGDP','NGDPD','NGDPRPC','NGDPPC','LUR','NGDPDPC','GGXWDG_NGDP','GGXWDG','GGXWDN_NGDP',\
                'GGXWDN','GGXCNL_NGDP','GGXCNL','GGXONLB_NGDP','GGXONLB','GGR_NGDP','GGR','GGSB_NPGDP','GGSB',\
                'GGX_NGDP','GGX','NGSD_NGDP','PPPEX','TM_RPCH','TMG_RPCH','PCPIPCH','PCPIEPCH','PCPI','PCPIE',\
                'NGAP_NPGDP','PPPPC','LP','PPPSH']

# https://www.imf.org/external/pubs/ft/weo/2017/01/weodata/download.aspx
# File: https://www.imf.org/external/pubs/ft/weo/2017/01/weodata/WEOApr2017all.xls
# Note that you have to convert manually .xls into .xlsx (save with excel)
filename="WEOApr2017all.xlsx"
folder="data/"

dataset = pd.read_excel(folder+filename, thousands=',', inplace=True)
droplist=["WEO Country Code","Estimates Start After","Country","Subject Descriptor","Subject Notes","Units", "Scale","Country/Series-specific Notes"]

dataset=dataset.drop(droplist, axis=1)  
dataset.set_index(['ISO'], inplace=True)

subset= pd.DataFrame()
header_list=dataset.columns.values
list_meta=["WEO Subject Code"]
list_year=list(set(header_list) - set(list_meta))
year=["year"]
year.extend(list_year)

for iso in iso3list_weo:
    subset=dataset.loc[iso]
    #transpose dataset
    subset=subset.T  
    subset=subset.replace('n/a', np.NaN)
    subset=subset.replace('--', np.NaN)
    subset.insert(0,'year',year)
    for cname,iso3 in country_dict.items():
        if iso3==iso : 
            subset.insert(0, 'iso3',iso3)
    subset.set_index(['iso3', 'year'])
    subset.columns = subset.iloc[0]
    subset=subset.rename(columns = {iso:'iso3'})
    subset=subset.drop(subset.index[0])
    subset = subset.reset_index(drop=True)
    subset.set_index(['iso3','year'], inplace=True)     
    subset.to_csv(folder+'imf/csv/'+iso+'_WEO.csv', encoding = 'utf8')
    
#=== Done: victory message and song ===
print("\n*******************# Job Well done ! #********************\n")
winsound.PlaySound("SystemExit", winsound.SND_ALIAS)


*******************# Job Well done ! #********************



### 2. Compute group averages

NB. This script as evolve, it is functional but it needs to be cleaned to avoid the extra steps of saving as csv (to erase multi index) I shouldn't have use multi indexing to begining with. Check if i can do otherwise.

In [4]:
import pandas as pd
from pandas import Series, DataFrame, concat
import numpy as np
import csv
import winsound

#working directory
folder="data/imf/csv/"

isolist=['ABW', 'AFG', 'AGO', 'ALB', 'AND', 'ARE', 'ARG', 'ARM', 'ASM', 'ATG', 'AUS', 'AUT', 'AZE', 'BDI', 'BEL', 'BEN', 'BFA', 'BGD', 'BGR', 'BHR', 'BHS', 'BIH', 'BLR', 'BLZ', 'BMU', 'BOL', 'BRA', 'BRB', 'BRN', 'BTN', 'BWA', 'CAF', 'CAN', 'CHE', 'CHL', 'CHN', 'CIV', 'CMR', 'COD', 'COG', 'COL', 'COM', 'CPV', 'CRI', 'CUB', 'CUW', 'CYM', 'CYP', 'CZE', 'DEU', 'DJI', 'DMA', 'DNK', 'DOM', 'DZA', 'ECU', 'EGY', 'ERI', 'ESP', 'EST', 'ETH', 'FIN', 'FJI', 'FRA', 'FRO', 'FSM', 'GAB', 'GBR', 'GEO', 'GHA', 'GIN', 'GMB', 'GNB', 'GNQ', 'GRC', 'GRD', 'GRL', 'GTM', 'GUM', 'GUY', 'HKG', 'HND', 'HRV', 'HTI', 'HUN', 'IDN', 'IMN', 'IND', 'IRL', 'IRN', 'IRQ', 'ISL', 'ISR', 'ITA', 'JAM', 'JOR', 'JPN', 'KAZ', 'KEN', 'KGZ', 'KHM', 'KNA', 'KOR', 'KWT', 'LAO', 'LBN', 'LBR', 'LBY', 'LCA', 'LIE', 'LKA', 'LSO', 'LTU', 'LUX', 'LVA', 'MAC', 'MAF', 'MAR', 'MCO', 'MDA', 'MDG', 'MDV', 'MEX', 'MHL', 'MKD', 'MLI', 'MLT', 'MMR', 'MNE', 'MNG', 'MNP', 'MOZ', 'MRT', 'MUS', 'MWI', 'MYS', 'NAM', 'NCL', 'NER', 'NGA', 'NIC', 'NLD', 'NOR', 'NPL', 'NZL', 'OMN', 'PAK', 'PAN', 'PER', 'PHL', 'PLW', 'PNG', 'POL', 'PRI', 'PRT', 'PRY', 'PSE', 'PYF', 'QAT', 'ROU', 'RUS', 'RWA', 'SAU', 'SDN', 'SEN', 'SGP', 'SLB', 'SLE', 'SLV', 'SMR', 'SOM', 'SRB', 'SSD', 'STP', 'SUR', 'SVK', 'SVN', 'SWE', 'SWZ', 'SXM', 'SYC', 'SYR', 'TCA', 'TCD', 'TGO', 'THA', 'TJK', 'TKM', 'TLS', 'TON', 'TTO', 'TUN', 'TUR', 'TUV', 'TWN', 'TZA', 'UGA', 'UKR', 'URY', 'USA', 'UZB', 'VCT', 'VEN', 'VIR', 'VNM', 'VUT', 'WSM', 'YEM', 'ZAF', 'ZMB', 'ZWE']
iso3list_weo=["AFG","ALB","DZA","AGO","ATG","ARG","ARM","AUS","AUT","AZE","BHR","BGD","BRB","BLR","BEL","BLZ","BEN","BTN","BOL","BIH","BWA","BRA","BRN","BGR","BFA","BDI","CPV","KHM","CMR","CAN","CAF","TCD","CHL","CHN","COL","COM","CRI","HRV","CYP","CZE","DNK","DJI","DMA","DOM","ECU","SLV","GNQ","ERI","EST","ETH","FJI","FIN","FRA","GAB","GEO","DEU","GHA","GRC","GRD","GTM","GIN","GNB","GUY","HTI","HND","HUN","ISL","IND","IDN","IRQ","IRL","ISR","ITA","JAM","JPN","JOR","KAZ","KEN","KIR","KWT","KGZ","LVA","LBN","LSO","LBR","LBY","LTU","LUX","MDG","MWI","MYS","MDV","MLI","MLT","MHL","MRT","MUS","MEX","MDA","MNG","MNE","MAR","MOZ","MMR","NAM","NPL","NLD","NZL","NIC","NER","NGA","NOR","OMN","PAK","PLW","PAN","PNG","PRY","PER","PHL","POL","PRT","PRI","QAT","ROU","RWA","WSM","SMR","SAU","SEN","SRB","SYC","SLE","SGP","SVK","SVN","SLB","ZAF","SSD","ESP","LKA","KNA","LCA","VCT","SDN","SUR","SWZ","SWE","CHE","TJK","TZA","THA","TLS","TGO","TON","TTO","TUN","TUR","TKM","TUV","UGA","UKR","ARE","GBR","USA","URY","UZB","VUT","YEM","VNM","ZMB","ZWE"]

#Region dictionnary that affect to a country iso3 its income group, region group, and worl:
region_dict= {"ASM":"UMC,EAS,WLD","AUS":"OEC,EAS,WLD","BRN":"NOC,EAS,WLD","CHN":"UMC,EAS,WLD","FJI":"UMC,EAS,WLD","FSM":"LMC,EAS,WLD","GUM":"NOC,EAS,WLD","HKG":"NOC,EAS,WLD","IDN":"LMC,EAS,WLD","JPN":"OEC,EAS,WLD","KHM":"LIC,EAS,WLD","KIR":"LMC,EAS,WLD","KOR":"OEC,EAS,WLD","LAO":"LMC,EAS,WLD","MAC":"NOC,EAS,WLD","MHL":"UMC,EAS,WLD","MMR":"LMC,EAS,WLD","MNG":"UMC,EAS,WLD","MNP":"NOC,EAS,WLD","MYS":"UMC,EAS,WLD","NCL":"NOC,EAS,WLD","NZL":"OEC,EAS,WLD","PHL":"LMC,EAS,WLD","PLW":"UMC,EAS,WLD","PNG":"LMC,EAS,WLD","PRK":"LIC,EAS,WLD","PYF":"NOC,EAS,WLD","SGP":"NOC,EAS,WLD","SLB":"LMC,EAS,WLD","THA":"UMC,EAS,WLD","TLS":"LMC,EAS,WLD","TON":"UMC,EAS,WLD","TUV":"UMC,EAS,WLD","TWN":"NOC,EAS,WLD","VNM":"LMC,EAS,WLD","VUT":"LMC,EAS,WLD","WSM":"LMC,EAS,WLD","":"LMC,ECS,WLD","ALB":"UMC,ECS,WLD","AND":"NOC,ECS,WLD","ARM":"LMC,ECS,WLD","AUT":"OEC,ECS,WLD","AZE":"UMC,ECS,WLD","BEL":"OEC,ECS,WLD","BGR":"UMC,ECS,WLD","BIH":"UMC,ECS,WLD","BLR":"UMC,ECS,WLD","CHE":"OEC,ECS,WLD","CHI":"NOC,ECS,WLD","CYP":"NOC,ECS,WLD","CZE":"OEC,ECS,WLD","DEU":"OEC,ECS,WLD","DNK":"OEC,ECS,WLD","ESP":"OEC,ECS,WLD","EST":"OEC,ECS,WLD","FIN":"OEC,ECS,WLD","FRA":"OEC,ECS,WLD","FRO":"NOC,ECS,WLD","GBR":"OEC,ECS,WLD","GEO":"LMC,ECS,WLD","GRC":"OEC,ECS,WLD","GRL":"NOC,ECS,WLD","HRV":"NOC,ECS,WLD","HUN":"OEC,ECS,WLD","IMN":"NOC,ECS,WLD","IRL":"OEC,ECS,WLD","ISL":"OEC,ECS,WLD","ITA":"OEC,ECS,WLD","KAZ":"UMC,ECS,WLD","KGZ":"LMC,ECS,WLD","LIE":"NOC,ECS,WLD","LTU":"NOC,ECS,WLD","LUX":"OEC,ECS,WLD","LVA":"NOC,ECS,WLD","MCO":"NOC,ECS,WLD","MDA":"LMC,ECS,WLD","MKD":"UMC,ECS,WLD","MNE":"UMC,ECS,WLD","NLD":"OEC,ECS,WLD","NOR":"OEC,ECS,WLD","POL":"OEC,ECS,WLD","PRT":"OEC,ECS,WLD","ROU":"UMC,ECS,WLD","RUS":"NOC,ECS,WLD","SMR":"NOC,ECS,WLD","SRB":"UMC,ECS,WLD","SVK":"OEC,ECS,WLD","SVN":"OEC,ECS,WLD","SWE":"OEC,ECS,WLD","TJK":"LMC,ECS,WLD","TKM":"UMC,ECS,WLD","TUR":"UMC,ECS,WLD","UKR":"LMC,ECS,WLD","UZB":"LMC,ECS,WLD","ABW":"NOC,LCN,WLD","ARG":"NOC,LCN,WLD","ATG":"NOC,LCN,WLD","BHS":"NOC,LCN,WLD","BLZ":"UMC,LCN,WLD","BOL":"LMC,LCN,WLD","BRA":"UMC,LCN,WLD","BRB":"NOC,LCN,WLD","CHL":"OEC,LCN,WLD","COL":"UMC,LCN,WLD","CRI":"UMC,LCN,WLD","CUB":"UMC,LCN,WLD","CUW":"NOC,LCN,WLD","CYM":"NOC,LCN,WLD","DMA":"UMC,LCN,WLD","DOM":"UMC,LCN,WLD","ECU":"UMC,LCN,WLD","GRD":"UMC,LCN,WLD","GTM":"LMC,LCN,WLD","GUY":"LMC,LCN,WLD","HND":"LMC,LCN,WLD","HTI":"LIC,LCN,WLD","JAM":"UMC,LCN,WLD","KNA":"NOC,LCN,WLD","LCA":"UMC,LCN,WLD","MAF":"NOC,LCN,WLD","MEX":"UMC,LCN,WLD","NIC":"LMC,LCN,WLD","PAN":"UMC,LCN,WLD","PER":"UMC,LCN,WLD","PRI":"NOC,LCN,WLD","PRY":"UMC,LCN,WLD","SLV":"LMC,LCN,WLD","SUR":"UMC,LCN,WLD","SXM":"NOC,LCN,WLD","TCA":"NOC,LCN,WLD","TTO":"NOC,LCN,WLD","URY":"NOC,LCN,WLD","VCT":"UMC,LCN,WLD","VEN":"NOC,LCN,WLD","VIR":"NOC,LCN,WLD","ARE":"NOC,MEA,WLD","BHR":"NOC,MEA,WLD","DJI":"LMC,MEA,WLD","DZA":"UMC,MEA,WLD","EGY":"LMC,MEA,WLD","IRN":"UMC,MEA,WLD","IRQ":"UMC,MEA,WLD","ISR":"OEC,MEA,WLD","JOR":"UMC,MEA,WLD","KWT":"NOC,MEA,WLD","LBN":"UMC,MEA,WLD","LBY":"UMC,MEA,WLD","MAR":"LMC,MEA,WLD","MLT":"NOC,MEA,WLD","OMN":"NOC,MEA,WLD","PSE":"LMC,MEA,WLD","QAT":"NOC,MEA,WLD","SAU":"NOC,MEA,WLD","SYR":"LMC,MEA,WLD","TUN":"UMC,MEA,WLD","YEM":"LMC,MEA,WLD","BMU":"NOC,NAC,WLD","CAN":"OEC,NAC,WLD","USA":"OEC,NAC,WLD","AFG":"LIC,SAS,WLD","BGD":"LMC,SAS,WLD","BTN":"LMC,SAS,WLD","IND":"LMC,SAS,WLD","LKA":"LMC,SAS,WLD","MDV":"UMC,SAS,WLD","NPL":"LIC,SAS,WLD","PAK":"LMC,SAS,WLD","AGO":"UMC,SSF,WLD","BDI":"LIC,SSF,WLD","BEN":"LIC,SSF,WLD","BFA":"LIC,SSF,WLD","BWA":"UMC,SSF,WLD","CAF":"LIC,SSF,WLD","CIV":"LMC,SSF,WLD","CMR":"LMC,SSF,WLD","COD":"LIC,SSF,WLD","COG":"LMC,SSF,WLD","COM":"LIC,SSF,WLD","CPV":"LMC,SSF,WLD","ERI":"LIC,SSF,WLD","ETH":"LIC,SSF,WLD","GAB":"UMC,SSF,WLD","GHA":"LMC,SSF,WLD","GIN":"LIC,SSF,WLD","GMB":"LIC,SSF,WLD","GNB":"LIC,SSF,WLD","GNQ":"NOC,SSF,WLD","KEN":"LMC,SSF,WLD","LBR":"LIC,SSF,WLD","LSO":"LMC,SSF,WLD","MDG":"LIC,SSF,WLD","MLI":"LIC,SSF,WLD","MOZ":"LIC,SSF,WLD","MRT":"LMC,SSF,WLD","MUS":"UMC,SSF,WLD","MWI":"LIC,SSF,WLD","NAM":"UMC,SSF,WLD","NER":"LIC,SSF,WLD","NGA":"LMC,SSF,WLD","RWA":"LIC,SSF,WLD","SDN":"LMC,SSF,WLD","SEN":"LMC,SSF,WLD","SLE":"LIC,SSF,WLD","SOM":"LIC,SSF,WLD","SSD":"LIC,SSF,WLD","STP":"LMC,SSF,WLD","SWZ":"LMC,SSF,WLD","SYC":"NOC,SSF,WLD","TCD":"LIC,SSF,WLD","TGO":"LIC,SSF,WLD","TZA":"LIC,SSF,WLD","UGA":"LIC,SSF,WLD","ZAF":"UMC,SSF,WLD","ZMB":"LMC,SSF,WLD","ZWE":"LIC,SSF,WLD"}

#====================== Generate IMF WEO complete dataset =====================
iso_len=len(iso3list_weo)
frame = dict()
#this concatenates all WEO-IMF csv files
i=1
for iso in iso3list_weo:
    isolistPlus=[]
    frame[i]=pd.read_csv(folder+iso+'_WEO.csv')
    
    # Add income class and Region 
    #browse the region dictionnary
    for iso3,region_iso in region_dict.items(): 
            if iso==iso3:
                isolistPlus.append(iso)
                for item in region_iso.split(','):
                    isolistPlus.append(str(item))
                    
    #save income class et Region in a separate variable
    frame[i]['Income_class']= isolistPlus[1]
    frame[i]['Region']= isolistPlus[2]
    #Add name columns names for col 1 and 2:  iso3 and year
    frame[i]=frame[i].rename(columns = {'Unnamed: 0':'iso3', 'Unnamed: 1':'year'})
    i+=1
    
#Concat all country files frame[1]...frame[n] 
final=DataFrame()
j=1

#concatenate all files
for iso in iso3list_weo:
    final = concat([final, frame[j]])
    j+=1
final.to_csv(folder+'final_WEO.csv')

#========================>   Computing averages    <=========================
#Compute average by income for all indicators in the DataFrame 'final' 
income= final.groupby(['Income_class', 'year']).agg([np.mean])
#Compute average by region for all indicators in the DataFrame 'final' 
region= final.groupby(['Region', 'year']).agg([np.mean])
#compute world average
world= final.groupby(['year']).agg([np.mean])

#!! world is not multiindexed here, it only has year, we thus need to create a multi index containing iso3= WLD
yearRange=[]
isolistW=[]

#Save as list dates ex 1980, 1981, ... 2020
yearRange=world.index.values
#Save as list  'WLD' time the number of values in world.index.values
for item in world.index.values:
    isolistW.append('WLD')

#Redindex World as (WLD,year) eg (WLD, 1980)...(WLD,2020)
world.index= list(zip(isolistW, yearRange))

#in a new DataFrame, concatenate income, region averages and world average
final_mean = concat([income,region,world])
final_mean.index.name = ['iso3', 'year']
#save it as a csv
final_mean.to_csv(folder+'WEO_mean.csv')
#==============================================< Append dataset and means >============================================
dataset = pd.read_csv(folder+'WEO_mean.csv')
dataset=dataset.rename(columns = {'Unnamed: 0':'iso3', 'Unnamed: 1':'year'})
dataset=dataset.drop(dataset.index[[0,1]])
dataset_weo=pd.read_csv(folder+'final_WEO.csv')
dataset=dataset.append(dataset_weo)
dataset = dataset.drop('Unnamed: 0', 1)
dataset.to_csv(folder+'final_WEO_with_avg.csv')

#====================================================< END >============================================================
print("\n*******************# Job Well done ! #********************\n")
winsound.PlaySound("SystemHand", winsound.SND_ALIAS)


*******************# Job Well done ! #********************



### 3. Write the jsons from CSVs

In [5]:
import pandas as pd
from pandas import Series, DataFrame, concat
from datetime import datetime
import numpy as np
import winsound
import os

#Working directory
folder="data/imf/csv/"
folder_final="data/imf/"

#Set begining and end of the serie
start=1980
end=2022

#=================================================================================================
indicator_list=['NID_NGDP','PPPGDP','BCA_NGDPD','BCA','LE','TX_RPCH','TXG_RPCH','NGDP_FY','NGDP_D','NGDP_RPCH','NGDP_R','NGDP','NGDPD','NGDPRPC','NGDPPC','LUR','NGDPDPC','GGXWDG_NGDP','GGXWDG','GGXWDN_NGDP','GGXWDN','GGXCNL_NGDP','GGXCNL','GGXONLB_NGDP','GGXONLB','GGR_NGDP','GGR','GGSB_NPGDP','GGSB','GGX_NGDP','GGX','NGSD_NGDP','PPPEX','TM_RPCH','TMG_RPCH','PCPIPCH','PCPIEPCH','PCPI','PCPIE','NGAP_NPGDP','PPPPC','LP','PPPSH']
iso3list=['ABW', 'AFG', 'AGO', 'ALB', 'AND', 'ARE', 'ARG', 'ARM', 'ASM', 'ATG', 'AUS', 'AUT', 'AZE', 'BDI', 'BEL', 'BEN', \
          'BFA', 'BGD', 'BGR', 'BHR', 'BHS', 'BIH', 'BLR', 'BLZ', 'BMU', 'BOL', 'BRA', 'BRB', 'BRN', 'BTN', 'BWA', 'CAF',\
          'CAN', 'CHE', 'CHL', 'CHN', 'CIV', 'CMR', 'COD', 'COG', 'COL', 'COM', 'CPV', 'CRI', 'CUB', 'CUW', 'CYM', 'CYP', \
          'CZE', 'DEU', 'DJI', 'DMA', 'DNK', 'DOM', 'DZA', 'ECU', 'EGY', 'ERI', 'ESP', 'EST', 'ETH', 'FIN', 'FJI', 'FRA', \
          'FRO', 'FSM', 'GAB', 'GBR', 'GEO', 'GHA', 'GIN', 'GMB', 'GNB', 'GNQ', 'GRC', 'GRD', 'GRL', 'GTM', 'GUM', 'GUY',\
          'HKG', 'HND', 'HRV', 'HTI', 'HUN', 'IDN', 'IMN', 'IND', 'IRL', 'IRN', 'IRQ', 'ISL', 'ISR', 'ITA', 'JAM', 'JOR', \
          'JPN', 'KAZ', 'KEN', 'KGZ', 'KHM', 'KNA', 'KOR', 'KWT', 'LAO', 'LBN', 'LBR', 'LBY', 'LCA', 'LIE', 'LKA', 'LSO', \
          'LTU', 'LUX', 'LVA', 'MAC', 'MAF', 'MAR', 'MCO', 'MDA', 'MDG', 'MDV', 'MEX', 'MHL', 'MKD', 'MLI', 'MLT', 'MMR',\
          'MNE', 'MNG', 'MNP', 'MOZ', 'MRT', 'MUS', 'MWI', 'MYS', 'NAM', 'NCL', 'NER', 'NGA', 'NIC', 'NLD', 'NOR', 'NPL',\
          'NZL', 'OMN', 'PAK', 'PAN', 'PER', 'PHL', 'PLW', 'PNG', 'POL', 'PRI', 'PRT', 'PRY', 'PSE', 'PYF', 'QAT', 'ROU', \
          'RUS', 'RWA', 'SAU', 'SDN', 'SEN', 'SGP', 'SLB', 'SLE', 'SLV', 'SMR', 'SOM', 'SRB', 'SSD', 'STP', 'SUR', 'SVK',\
          'SVN', 'SWE', 'SWZ', 'SXM', 'SYC', 'SYR', 'TCA', 'TCD', 'TGO', 'THA', 'TJK', 'TKM', 'TLS', 'TON', 'TTO', 'TUN',\
          'TUR', 'TUV', 'TWN', 'TZA', 'UGA', 'UKR', 'URY', 'USA', 'UZB', 'VCT', 'VEN', 'VIR', 'VNM', 'VUT', 'WSM', 'YEM', \
          'ZAF', 'ZMB', 'ZWE']

#This dictionnary associate a country name to an iso3 code 
#Region dictionnary that affect to a country iso3 its income group, region group, and worl:
region_dict= {"ASM":"UMC,EAS,WLD","AUS":"OEC,EAS,WLD","BRN":"NOC,EAS,WLD","CHN":"UMC,EAS,WLD","FJI":"UMC,EAS,WLD","FSM":"LMC,EAS,WLD","GUM":"NOC,EAS,WLD","HKG":"NOC,EAS,WLD","IDN":"LMC,EAS,WLD","JPN":"OEC,EAS,WLD","KHM":"LIC,EAS,WLD","KIR":"LMC,EAS,WLD","KOR":"OEC,EAS,WLD","LAO":"LMC,EAS,WLD","MAC":"NOC,EAS,WLD","MHL":"UMC,EAS,WLD","MMR":"LMC,EAS,WLD","MNG":"UMC,EAS,WLD","MNP":"NOC,EAS,WLD","MYS":"UMC,EAS,WLD","NCL":"NOC,EAS,WLD","NZL":"OEC,EAS,WLD","PHL":"LMC,EAS,WLD","PLW":"UMC,EAS,WLD","PNG":"LMC,EAS,WLD","PRK":"LIC,EAS,WLD","PYF":"NOC,EAS,WLD","SGP":"NOC,EAS,WLD","SLB":"LMC,EAS,WLD","THA":"UMC,EAS,WLD","TLS":"LMC,EAS,WLD","TON":"UMC,EAS,WLD","TUV":"UMC,EAS,WLD","TWN":"NOC,EAS,WLD","VNM":"LMC,EAS,WLD","VUT":"LMC,EAS,WLD","WSM":"LMC,EAS,WLD","":"LMC,ECS,WLD","ALB":"UMC,ECS,WLD","AND":"NOC,ECS,WLD","ARM":"LMC,ECS,WLD","AUT":"OEC,ECS,WLD","AZE":"UMC,ECS,WLD","BEL":"OEC,ECS,WLD","BGR":"UMC,ECS,WLD","BIH":"UMC,ECS,WLD","BLR":"UMC,ECS,WLD","CHE":"OEC,ECS,WLD","CHI":"NOC,ECS,WLD","CYP":"NOC,ECS,WLD","CZE":"OEC,ECS,WLD","DEU":"OEC,ECS,WLD","DNK":"OEC,ECS,WLD","ESP":"OEC,ECS,WLD","EST":"OEC,ECS,WLD","FIN":"OEC,ECS,WLD","FRA":"OEC,ECS,WLD","FRO":"NOC,ECS,WLD","GBR":"OEC,ECS,WLD","GEO":"LMC,ECS,WLD","GRC":"OEC,ECS,WLD","GRL":"NOC,ECS,WLD","HRV":"NOC,ECS,WLD","HUN":"OEC,ECS,WLD","IMN":"NOC,ECS,WLD","IRL":"OEC,ECS,WLD","ISL":"OEC,ECS,WLD","ITA":"OEC,ECS,WLD","KAZ":"UMC,ECS,WLD","KGZ":"LMC,ECS,WLD","LIE":"NOC,ECS,WLD","LTU":"NOC,ECS,WLD","LUX":"OEC,ECS,WLD","LVA":"NOC,ECS,WLD","MCO":"NOC,ECS,WLD","MDA":"LMC,ECS,WLD","MKD":"UMC,ECS,WLD","MNE":"UMC,ECS,WLD","NLD":"OEC,ECS,WLD","NOR":"OEC,ECS,WLD","POL":"OEC,ECS,WLD","PRT":"OEC,ECS,WLD","ROU":"UMC,ECS,WLD","RUS":"NOC,ECS,WLD","SMR":"NOC,ECS,WLD","SRB":"UMC,ECS,WLD","SVK":"OEC,ECS,WLD","SVN":"OEC,ECS,WLD","SWE":"OEC,ECS,WLD","TJK":"LMC,ECS,WLD","TKM":"UMC,ECS,WLD","TUR":"UMC,ECS,WLD","UKR":"LMC,ECS,WLD","UZB":"LMC,ECS,WLD","ABW":"NOC,LCN,WLD","ARG":"NOC,LCN,WLD","ATG":"NOC,LCN,WLD","BHS":"NOC,LCN,WLD","BLZ":"UMC,LCN,WLD","BOL":"LMC,LCN,WLD","BRA":"UMC,LCN,WLD","BRB":"NOC,LCN,WLD","CHL":"OEC,LCN,WLD","COL":"UMC,LCN,WLD","CRI":"UMC,LCN,WLD","CUB":"UMC,LCN,WLD","CUW":"NOC,LCN,WLD","CYM":"NOC,LCN,WLD","DMA":"UMC,LCN,WLD","DOM":"UMC,LCN,WLD","ECU":"UMC,LCN,WLD","GRD":"UMC,LCN,WLD","GTM":"LMC,LCN,WLD","GUY":"LMC,LCN,WLD","HND":"LMC,LCN,WLD","HTI":"LIC,LCN,WLD","JAM":"UMC,LCN,WLD","KNA":"NOC,LCN,WLD","LCA":"UMC,LCN,WLD","MAF":"NOC,LCN,WLD","MEX":"UMC,LCN,WLD","NIC":"LMC,LCN,WLD","PAN":"UMC,LCN,WLD","PER":"UMC,LCN,WLD","PRI":"NOC,LCN,WLD","PRY":"UMC,LCN,WLD","SLV":"LMC,LCN,WLD","SUR":"UMC,LCN,WLD","SXM":"NOC,LCN,WLD","TCA":"NOC,LCN,WLD","TTO":"NOC,LCN,WLD","URY":"NOC,LCN,WLD","VCT":"UMC,LCN,WLD","VEN":"NOC,LCN,WLD","VIR":"NOC,LCN,WLD","ARE":"NOC,MEA,WLD","BHR":"NOC,MEA,WLD","DJI":"LMC,MEA,WLD","DZA":"UMC,MEA,WLD","EGY":"LMC,MEA,WLD","IRN":"UMC,MEA,WLD","IRQ":"UMC,MEA,WLD","ISR":"OEC,MEA,WLD","JOR":"UMC,MEA,WLD","KWT":"NOC,MEA,WLD","LBN":"UMC,MEA,WLD","LBY":"UMC,MEA,WLD","MAR":"LMC,MEA,WLD","MLT":"NOC,MEA,WLD","OMN":"NOC,MEA,WLD","PSE":"LMC,MEA,WLD","QAT":"NOC,MEA,WLD","SAU":"NOC,MEA,WLD","SYR":"LMC,MEA,WLD","TUN":"UMC,MEA,WLD","YEM":"LMC,MEA,WLD","BMU":"NOC,NAC,WLD","CAN":"OEC,NAC,WLD","USA":"OEC,NAC,WLD","AFG":"LIC,SAS,WLD","BGD":"LMC,SAS,WLD","BTN":"LMC,SAS,WLD","IND":"LMC,SAS,WLD","LKA":"LMC,SAS,WLD","MDV":"UMC,SAS,WLD","NPL":"LIC,SAS,WLD","PAK":"LMC,SAS,WLD","AGO":"UMC,SSF,WLD","BDI":"LIC,SSF,WLD","BEN":"LIC,SSF,WLD","BFA":"LIC,SSF,WLD","BWA":"UMC,SSF,WLD","CAF":"LIC,SSF,WLD","CIV":"LMC,SSF,WLD","CMR":"LMC,SSF,WLD","COD":"LIC,SSF,WLD","COG":"LMC,SSF,WLD","COM":"LIC,SSF,WLD","CPV":"LMC,SSF,WLD","ERI":"LIC,SSF,WLD","ETH":"LIC,SSF,WLD","GAB":"UMC,SSF,WLD","GHA":"LMC,SSF,WLD","GIN":"LIC,SSF,WLD","GMB":"LIC,SSF,WLD","GNB":"LIC,SSF,WLD","GNQ":"NOC,SSF,WLD","KEN":"LMC,SSF,WLD","LBR":"LIC,SSF,WLD","LSO":"LMC,SSF,WLD","MDG":"LIC,SSF,WLD","MLI":"LIC,SSF,WLD","MOZ":"LIC,SSF,WLD","MRT":"LMC,SSF,WLD","MUS":"UMC,SSF,WLD","MWI":"LIC,SSF,WLD","NAM":"UMC,SSF,WLD","NER":"LIC,SSF,WLD","NGA":"LMC,SSF,WLD","RWA":"LIC,SSF,WLD","SDN":"LMC,SSF,WLD","SEN":"LMC,SSF,WLD","SLE":"LIC,SSF,WLD","SOM":"LIC,SSF,WLD","SSD":"LIC,SSF,WLD","STP":"LMC,SSF,WLD","SWZ":"LMC,SSF,WLD","SYC":"NOC,SSF,WLD","TCD":"LIC,SSF,WLD","TGO":"LIC,SSF,WLD","TZA":"LIC,SSF,WLD","UGA":"LIC,SSF,WLD","ZAF":"UMC,SSF,WLD","ZMB":"LMC,SSF,WLD","ZWE":"LIC,SSF,WLD"}
#=================================================================================================

dataset = pd.read_csv(folder+'final_WEO_with_avg.csv')

for ISO in iso3list_weo:
    print("Proceed: ",ISO)
    if dataset.empty: continue
    isolistPlus=[] 
    #browse the region dictionnary
    for iso3,region_iso in region_dict.items(): 
            if ISO==iso3:
                isolistPlus.append(ISO)
                for item in region_iso.split(','):
                    isolistPlus.append(str(item))
                    
    print(isolistPlus)
    #================== store the meta information ====================================================
    source='"data_source":{"source":"IMF, WEO", "provider":"https://www.imf.org/", "Download_date":"'+datetime.now().strftime('%Y-%m-%d')+'" }'
    #read country meta stored in a text file
    with open (folder_final+"json meta country.json.txt", "r") as meta:
        for line in meta :
            if line.startswith('"'+ISO+'"') :
                metafinal=","+line+","+source
    #=================================================================================================


    #store name of the last ind in the list
    last=indicator_list[len(indicator_list)-1]

    file = open(folder_final+"temp.json", "w")
    #Start writing the json file:
    file.write('{')

    #Loop over indicator list
    for ind in indicator_list:
        
        #Get indicator meta from external file
        filemeta=open(folder+"meta indicator IMF.txt", "r")
        for line in filemeta :
            if line.startswith('"'+ind+'"') : meta=line
        filemeta.close()
        #write the meta
        file.write(meta)

        #loop over the isolistPlus which contain [countryiso,incomegrp, regionalgrp, world] 
        for iso in isolistPlus:
            #write iso name
            file.write('"'+iso+'":{')  
            #set min and max year for each ind and iso
            year_range=dataset["year"][(dataset["iso3"]==iso)&(dataset[ind].notnull())]
            if len(year_range)!=0:
                Ymin=min(year_range)
                Ymax=max(year_range)
            else:
                Ymin=Ymax="null"
            #write this information within the serie        
            file.write('"YearMin":'+str(Ymin)+',')
            file.write('"YearMax":'+str(Ymax)+',')
            for year in range(start,end+1):
                x= dataset[ind][(dataset["year"] == year) & (dataset["iso3"]==iso)]
                if x.empty :  
                    x="null"
                else :
                    x=str('{:0.3f}'.format(float(x)))
                if ind!=last:
                    if year!=int(end) : file.write('"'+str(year)+'":'+str(x)+',')
                    if (iso!="WLD") & (year==int(end)): file.write('"'+str(year)+'":'+str(x)+'},')
                    if (iso=="WLD") & (year==int(end)): file.write('"'+str(year)+'":'+str(x)+'}},')
                if ind==last:
                    if year!=int(end) : file.write('"'+str(year)+'":'+str(x)+',')
                    if (iso!="WLD") & (year==int(end)): file.write('"'+str(year)+'":'+str(x)+'},')
                    if (iso=="WLD") & (year==int(end)): file.write('"'+str(year)+'":'+str(x)+'}}'+metafinal+'}')

    file.close()
    file_in = open(folder_final+"temp.json", "r")
    file_out = open(folder_final+"IMF_WEO_"+ISO+".json", "wt")
    for line in file_in: file_out.write( line.replace(':nan', ":null") )
    file_in.close()
    file_out.close()

    #os.remove(folder+'dataset'+ISO+'.csv')
    print("done:IMF_WEO_"+ISO+".json")
    print(datetime.now().strftime('%Y-%m-%d'))
print("\n*******************# Job Well done ! #********************\n")
#os.remove(folder_final+"temp.json")
winsound.PlaySound("SystemHand", winsound.SND_ALIAS)

Proceed:  AFG
['AFG', 'LIC', 'SAS', 'WLD']
done:IMF_WEO_AFG.json
2017-05-18
Proceed:  ALB
['ALB', 'UMC', 'ECS', 'WLD']


KeyboardInterrupt: 